In [86]:
import pandas as pd
import numpy as np


In [87]:
df = pd.read_csv('/Users/kunalkoshta/Desktop/MLOps/MLOps-Vehicle-Insurance/notebooks/data.csv')

In [88]:
df.shape

(381109, 12)

In [89]:
from sklearn.model_selection import train_test_split

In [90]:
train_df,test_df = train_test_split(df,test_size=0.2)

In [91]:
input_feature_train_df = train_df.drop(columns=['Response'], axis=1)
target_feature_train_df = train_df['Response']

In [92]:
input_feature_test_df = test_df.drop(columns=['Response'], axis=1)
target_feature_test_df = test_df['Response']

In [93]:
input_feature_train_df['Gender'] = input_feature_train_df['Gender'].map({"Male":1,"Female":0}).astype(int)

In [94]:
input_feature_train_df=pd.get_dummies(input_feature_train_df,drop_first=True)

In [95]:
input_feature_train_df.drop(columns=['id'], inplace=True)

In [96]:
input_feature_train_df = input_feature_train_df.rename(columns={
            "Vehicle_Age_< 1 Year": "Vehicle_Age_lt_1_Year",
            "Vehicle_Age_> 2 Years": "Vehicle_Age_gt_2_Years"
        })

In [97]:
input_feature_test_df['Gender'] = input_feature_test_df['Gender'].map({"Male":1,"Female":0}).astype(int)

In [98]:
input_feature_test_df=pd.get_dummies(input_feature_test_df,drop_first=True)


In [99]:
input_feature_test_df.drop(columns=['id'], inplace=True)


In [113]:
input_feature_test_df = input_feature_test_df.rename(columns={
            "Vehicle_Age_< 1 Year": "Vehicle_Age_lt_1_Year",
            "Vehicle_Age_> 2 Years": "Vehicle_Age_gt_2_Years"
        })

In [114]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.compose import ColumnTransformer

In [115]:
num_features = ['Age','Vintage']
mm_features = ['Annual_Premium']
numeric_transformer = StandardScaler()
mm_transformer = MinMaxScaler()

In [116]:
preprocessor = ColumnTransformer(
                transformers=[
                    ("StandardScaler",numeric_transformer,num_features),
                    ("MinMaxScaler",mm_transformer,mm_features)
                ],
                remainder='passthrough' #Leave other columns as they are
            )

In [117]:
from sklearn.pipeline import Pipeline

In [118]:
final_pipeline = Pipeline(steps=[("Preprocessor",preprocessor)])

In [119]:
input_feature_test_df

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Vehicle_Age_lt_1_Year,Vehicle_Age_gt_2_Years,Vehicle_Damage_Yes
110476,0,23,1,1.0,0,2630.0,160.0,34,True,False,True
9267,1,47,1,45.0,0,32011.0,26.0,20,False,False,True
96748,1,29,1,3.0,0,27736.0,124.0,261,False,False,True
56013,1,49,1,28.0,0,32155.0,26.0,68,False,False,True
280390,0,22,1,46.0,1,26403.0,152.0,13,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...
21966,1,52,1,28.0,0,31662.0,156.0,23,False,False,True
177062,0,35,1,46.0,1,24750.0,152.0,174,False,False,False
256812,0,24,1,29.0,1,29409.0,152.0,174,True,False,False
145495,1,63,1,28.0,0,38388.0,163.0,108,False,False,True


In [120]:
input_feature_train_arr = preprocessor.fit_transform(input_feature_train_df)

In [121]:
input_feature_test_arr = preprocessor.transform(input_feature_test_df)

In [122]:
from imblearn.combine import SMOTEENN
smt = SMOTEENN(sampling_strategy = 'minority')

In [123]:
input_feature_train_final, target_feature_train_final = smt.fit_resample(
                input_feature_train_arr,target_feature_train_df
            )

In [124]:
input_feature_test_final, target_feature_test_final = smt.fit_resample(
                input_feature_test_arr,target_feature_test_df
            )

In [125]:
train_arr = np.c_[input_feature_train_final,np.array(target_feature_train_final)]
test_arr = np.c_[input_feature_test_final,np.array(target_feature_test_final)]

In [148]:
x_train, y_train, x_test, x_test = train_arr[:,:-1], train_arr[:,-1], test_arr[:,:-1], test_arr[:,-1]

In [128]:
from sklearn.ensemble import RandomForestClassifier

In [129]:
model = RandomForestClassifier(
    n_estimators = 300,
    min_samples_split = 7,
    min_samples_leaf = 6,
    max_depth = 10,
    criterion = 'entropy',
    random_state = 2
)

In [130]:
model.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_leaf=6,
                       min_samples_split=7, n_estimators=300, random_state=2)

In [152]:
model.predict(x_test.reshape(1,-1))

ValueError: X has 101208 features, but RandomForestClassifier is expecting 11 features as input.